### Prob 7

We find the [LU decomposition](https://en.wikipedia.org/wiki/LU_decomposition) of matrix $A$ by running [Gaussian elimination](https://en.wikipedia.org/wiki/Gaussian_elimination) on $A$. The reduced matrix is the upper triangular matrix $U$, where the reduction is represented by left multiplication of transformation matrices $E_i$ for $i \in\{1,\ldots, n-1\}$,

$$U = \left(\overset{\curvearrowleft}{\prod_{i=1}^{n-1}} E_i\right) A = E_{n-1}\cdots E_1 A.$$

Further, it can be shown that 

$$L = \overset{\curvearrowright}{\prod_{i=1}^{n-1}} E_i^{-1} = E_1^{-1}\cdots E_{n-1}^{-1}$$

is a lower triangular matrix. Whence we obtain $A = LU$.

In [ ]:
A = matrix(QQ, [[4,1,0,0], [1,4,1,0], [0,1,4,1], [0,0,1,4]])
show(A)

In [ ]:
L = matrix(QQ, [[1,0,0,0],[1/4,1,0,0],[0,4/15,1,0],[0,0,15/56,1]])
show(L)

In [ ]:
U = matrix(QQ, [[4,1,0,0],[0,15/4,1,0],[0,0,56/15,1],[0,0,0,209/56]])
show(U)

In [ ]:
A == L*U

In [ ]:
b = vector(QQ, [2,-3,3,-2])
y = L.solve_right(b)
show(y)

In [ ]:
x = U.solve_right(y)
show(x)

In [ ]:
# to wit
L*y == b

In [ ]:
U*x == y

In [ ]:
A*x == b

### Prob 8

def wellposed(U,b):
    if not all(len(U[i]) == len(U) for i in range(len(U))):
        print "Matrix U is not square."; return False
    if not len(U[0]) == len(b):
        print "Vector b is not of compatible dimension with matrix U."; return False
    if not all(U[i][j] == 0 for i in range(len(U)) for j in range(i)): 
        print "Matrix U is not upper triangular."; return False
    if not all(U[i][i] != 0 for i in range(len(U))): 
        print "Matrix U has zero'd out pivot elements."; return False
    return True

def backsub(U,b):
    if not wellposed(U,b):
        return "Back substitution is ill-posed."
    x = []
    for i in range(len(b)):
        offset = sum(U[-(i+1)][-(j+1)]*x[-(j+1)] for j in range(i))
        elem = (b[-(i+1)] - offset)/U[-(i+1)][-(i+1)]
        x.insert(0, elem)
    return x

In [ ]:
# for example 
U = [[1,2],[0,1]]
b = [3,1]
backsub(U,b)

In [ ]:
# ill-posed example
U = [[2,1],[0,0]]
backsub(U,b)

In [ ]:
# desideratum
U = [[1,2,-1],[0,3,-1],[0,0,2]]
b = [-1, 0, 1]
backsub(U,b)

In [ ]:
# checking work with SageMath's matrix notation
U = matrix(QQ, [[1,2,-1],[0,3,-1],[0,0,2]])
b = vector(QQ, [-1, 0, 1])
U.solve_right(b)

In [ ]:
def wellposed(L,b):
    if not all(len(L[i]) == len(L) for i in range(len(L))):
        print "Matrix L is not square."; return False
    if not len(L[0]) == len(b):
        print "Vector b is not of compatible dimension with matrix L."; return False
    if not all(L[i][j] == 0 for j in range(len(L)) for i in range(j)): 
        print "Matrix L is not lower triangular."; return False
    if not all(L[i][i] != 0 for i in range(len(L))): 
        print "Matrix L has zero'd out pivot elements."; return False
    return True

def forwardsub(L,b):
    if not wellposed(L,b):
        return "Forward substitution is ill-posed."
    x = []
    for i in range(len(b)):
        offset = sum(L[i][j]*x[j] for j in range(i))
        elem = (b[i] - offset)/L[i][i]
        x.append(elem)
    return x

In [ ]:
# for example (the transpose of the previous)
L = [[1,0],[2,1]]
b = [1,3]
forwardsub(L,b)

In [ ]:
# ill-posed
L = [[1,9],[0,1]]
b = [1,3]
forwardsub(L,b)

In [ ]:
# desideratum
L = [[1,0,0],[2,1,0],[3,4,1]]
b = [-1, 0, 1]
forwardsub(L,b)

In [ ]:
# checking work with SageMath's matrix notation
L = matrix(QQ, [[1,0,0],[2,1,0],[3,4,1]])
b = vector(QQ, [-1, 0, 1])
L.solve_right(b)

### Prob 9

Suppose that $A$ is a tridiagonal $n\times n$ matrix with all pivot elements nonzero. Consider the linear system $A\mathbf{x} = \mathbf{b}$.

Gaussian Elimination takes $n-1$ passes to reduce the augmented matrix $[A|\mathbf{b}]$ to $[A'|\mathbf{b}']$ where $A'$ is upper triangular. In term, back substitution takes $n$ passes to produce the vector $\mathbf{x}$ from $\mathbf{b}'$ and $A'$.

Then, what is the order $O(n^p)$ of arithmetical operations required for the solution of $A\mathbf{x} = \mathbf{b}$ by Gaussian Elimination and back substitution? We'll count division and subtraction as $*$ and $+$ operations, respectively. 

Consider the structure of the matrix $[A|\mathbf{b}]$. I claim the $i$th pass of GE on $[A|\mathbf{b}]$ requires $3$ multiplications and $2$ additions.

$$\begin{align}
m_{i+1,i} &\leftarrow \frac{a_{i+1,i}}{a_{ii}}\\
a_{i+1,i+1}' &\leftarrow a_{i+1,i+1} - a_{i,i+1}*m_{i+1,i}\\
b_{i+1}' &\leftarrow b_{i+1} - b_i*m_{i+1,i}
\end{align}$$

where we need neither compute

$$\begin{align}
a_{i+1,i}' &\leftarrow a_{i+1,i} - a_{ii}*m_{i+1,i}\\
a_{i+1,j}' &\leftarrow a_{i+1,j} - a_{i,j}*m_{i+1,i} &\text{for $j > i+1$}
\end{align}$$ 

since we *know* $a_{i+1,i}' \leftarrow 0$ and $a_{i+1,j}' \leftarrow a_{i+1,j}$. (That is, $a_{i+1,i} - a_{ii}*m_{i+1,i} = 0$ by design of GE, and $a_{i,j} = 0$ for all $j > i+1$ by the tridiagonal structure of $A$.)

In turn, the $i$th pass ($i \in \{n, \ldots, 1\}$, decrementing) of back substitution requires $2$ multiplications and $1$ addition.

$$x_{i} \leftarrow \frac{b'_{i} - a_{i, i+1}*x_{i+1}}{a_{ii}}$$

where $a_{n,n+1}$ and $x_{n+1}$ are undefined (so we take $1$ addition and $1$ multiplication out of the whole count).

In total, the count of multiplications is 
$$GE(*) + RS(*) = 3n-3+2n-1 = 5n-4$$
and the count of additions is
$$GE(+) + RS(+) = 2n-2+n-1 = 3n-3$$

The order of arithmetical operations for Gaussian elimination and back substitution is thus $O(n)$ for tridiagonal matrices of size $n\times n$.
